In [1]:
# MEJNw2yQ
import pandas as pd
import requests
import glob
import os

import geopandas as gpd
# import folium
from scipy.interpolate import griddata
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

from thermal import Params, simulate_day

In [ ]:
files = glob.glob('./ghcn_hourly_data/GHCNh_*.parquet')
print(f"Found {len(files)} files")

columns = [
  "Station_ID",
  # "Station_name",
  "DATE",
  "Latitude",
  "Longitude",
  "Elevation",
  "temperature",
  "wind_speed",
  "relative_humidity",
  # "wet_bulb_temperature",
  # "altimeter",
  # "precipitation"
]

N = len(files)

df_stations = []
for i in range(N):
  if i % 100 == 0:
    print(f"Processing file {i} of {N}")
  df = pd.read_parquet(files[i], engine="pyarrow", columns=columns)

  # Interpolate or aggregate the DATE column to get only hourly data
  df['DATE'] = pd.to_datetime(df['DATE'])
  df.set_index('DATE', inplace=True)
  df_hourly = df.resample('1h').mean(numeric_only=True).reset_index()
  df_hourly["Station_ID"] = df.iloc[0]["Station_ID"]
  df_stations.append(df_hourly)

df_stations = pd.concat(df_stations)
df_stations.to_parquet("./ghcn_hourly_combined.parquet")

In [40]:
df = pd.read_parquet("./ghcn_hourly_combined.parquet")
df.rename(columns={'Latitude': 'latitude', 'Longitude': 'longitude', 'Elevation': 'elevation', 'DATE': 'ds'}, inplace=True)
df["month"] = df["ds"].dt.month
df["hour"] = df["ds"].dt.hour

In [ ]:
print(f"There are {df["Station_ID"].nunique()} unique locations")
num_stations = df["Station_ID"].nunique()
print(f"There are {len(df) / num_stations} measurements per station on average")

# For each station ID, impute missing hourly data with the mean of the other hours.
# df.temperature 

# Drop any stations with less than 80% complete data (0.80 * 8760 = 7008)
station_counts = df.groupby("Station_ID").size().reset_index(name="count")
station_counts

min_obs = 8760 * 0.80
df = df[df["Station_ID"].isin(station_counts[station_counts["count"] >= min_obs]["Station_ID"])]

print(f"After dropping stations with less than {min_obs} observations, there are {df['Station_ID'].nunique()} stations remaining")

# Now forward fill missing data
df = df.sort_values(by=["Station_ID", "ds"])
df = df.groupby("Station_ID").apply(lambda x: x.ffill()).reset_index(drop=True)

df

In [ ]:
# df.to_csv("./ghcn_hourly_combined_cleaned.csv")
df.to_parquet("./ghcn_hourly_combined_cleaned.parquet")

In [ ]:
# center_lat = df['latitude'].mean()
# center_lon = df['longitude'].mean()
# m = folium.Map(location=[center_lat, center_lon], zoom_start=10)

# for idx, row in df.iterrows():
#   folium.CircleMarker(
#     location=[row['latitude'], row['longitude']],
#     radius=8,
#     popup=f"Temperature: {row['temperature']}°C",
#     color='red',
#     fill=True,
#     fill_color='red'
#   ).add_to(m)
# # View the map
# m
print("Latitude range:", df['latitude'].min(), df['latitude'].max())
print("Longitude range:", df['longitude'].min(), df['longitude'].max())

fig, ax = plt.subplots(figsize=(12, 8), subplot_kw={'projection': ccrs.PlateCarree()})

# Add map features
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS, linestyle=':')

# Set map extent with some padding
# padding = 1  # degrees
# ax.set_extent([
#   df['longitude'].min() - padding,
#   df['longitude'].max() + padding,
#   df['latitude'].min() - padding,
#   df['latitude'].max() + padding
# ])

df_avg_temp = df.groupby(by=["latitude", "longitude"]).mean().reset_index()

# Simple scatter plot of points
scatter = ax.scatter(df_avg_temp['longitude'], df_avg_temp['latitude'],
                    c=df_avg_temp['temperature'],
                    cmap='RdYlBu_r',
                    s=100)
plt.colorbar(scatter, label='Temperature (°C)')

fig.show()

In [45]:
# For each location and month, calculate the average temperature at each hour.
df_cleaned = pd.read_parquet("./ghcn_hourly_combined_cleaned.parquet")

df_avg_temp = df_cleaned.groupby(by=["latitude", "longitude", "month", "hour"]).agg({
  "temperature": "mean",
  "wind_speed": "mean",
  "relative_humidity": "mean",
  "elevation": "mean",
  "Station_ID": "first",
}).reset_index()

# Drop any station IDs with less than 12*24 observations
df_avg_temp.to_parquet("./ghcn_avg_temp.parquet")

In [ ]:
# Plot an average day for each month for a sample station
station_id = df_avg_temp["Station_ID"].unique()[1000]
print(f"Plotting average day for station {station_id}")

df_avg_temp_station = df_avg_temp[df_avg_temp["Station_ID"] == station_id].copy()

# Shift hours based on the longitude
num_hours_offset = int(df_avg_temp_station["longitude"].iloc[0] / 15)
print(f"Shifting hours by {num_hours_offset} hours")
if num_hours_offset < 0:
  num_hours_offset = 24 + num_hours_offset

df_avg_temp_station["hour"] = (df_avg_temp_station["hour"] + num_hours_offset) % 24

df_avg_temp_station = df_avg_temp_station.sort_values(by=["month", "hour"]).reset_index()

for month in df_avg_temp_station["month"].unique():
  df_month = df_avg_temp_station[df_avg_temp_station["month"] == month]
  plt.plot(df_month["hour"], df_month["temperature"], label=f"Month {month}")

plt.legend()
plt.title(f"Average Day for Station {station_id}")
plt.xlabel("Hour")
plt.ylabel("Temperature (°C)")
plt.show()


In [ ]:
def process_station(df_station: pd.DataFrame, params: Params, target_temp: float) -> tuple[pd.DataFrame, pd.DataFrame]:
  # Shift hours based on the longitude
  num_hours_offset = int(df_station["longitude"].iloc[0] / 15)
  if num_hours_offset < 0:
    num_hours_offset = 24 + num_hours_offset

  # print(f"Shifting hours by {num_hours_offset} hours")
  df_station["hour"] = (df_station["hour"] + num_hours_offset) % 24

  df_results = []
  df_hourly_results = []
  for month in df_station["month"].unique():
    df_month = df_station[df_station["month"] == month]
    # Use the middle day of the month as the day of the year
    day_of_year = df_month["month"].iloc[0] / 12 * 365 + 15
    results = simulate_day(params, df_month["temperature"].tolist(), target_temp=target_temp, day_of_year=day_of_year)
    df_results.append({
      "station_id": station_id,
      "month": month,
      "day_of_year": day_of_year,
      "target_temp": target_temp,
      "cooling_load_kwh": results['total_cooling_load_kwh'],
      "heating_load_kwh": results['total_heating_load_kwh'],
      "n_cooling_hours": len(results['daily_cooling_hours']),
      "n_heating_hours": len(results['daily_heating_hours']),
    })
    df_hourly = pd.DataFrame(results['hourly_results'])
    df_hourly["station_id"] = station_id
    df_hourly["month"] = month
    df_hourly["day_of_year"] = day_of_year
    df_hourly["target_temp"] = target_temp
    df_hourly["hour"] = range(24)
    df_hourly_results.append(df_hourly)

  return pd.DataFrame(df_results), pd.concat(df_hourly_results)


df_avg_temp = pd.read_parquet("./ghcn_avg_temp.parquet")

station_ids = df_avg_temp["Station_ID"].unique()
print(f"There are {len(station_ids)} unique stations")

params = Params(
  u_value=4.0,            # W/m²·K
  height=4.0,             # m
  infiltration_rate=0.5,  # air changes per hour
  thermal_mass=100000,    # J/m²·K
  glazing_transmittance=0.8,  # fraction
  latitude=0,            # degrees North
)

target_temp = 23
# station_ids = station_ids[:10]

# for station_id in station_ids:
# station_id = station_ids[0]
df_thermal_summary = []
for i, station_id in enumerate(station_ids):
  if i % 100 == 0:
    print(f"Processing station {station_id} ({i}/{len(station_ids)})")
  df_station = df_avg_temp[df_avg_temp["Station_ID"] == station_id].sort_values(by=["month", "hour"])

  # Make sure there are no NaN temperatures:
  if df_station["temperature"].isna().any():
    raise ValueError(f"Station {station_id} has NaN temperatures")

  df_summary_results, df_hourly_results = process_station(df_station, params, target_temp)

  df_thermal_summary.append(df_summary_results)

df_thermal_summary = pd.concat(df_thermal_summary)

df_thermal_summary.to_parquet("./ghcn_station_summaries.parquet")
# df_hourly_results

In [ ]:
# Plot the hourly results for a sample station
df_plot = df_summary_results.copy()
station_id = df_plot["station_id"].iloc[0]

kwh_per_mmbtu = 293.07
electricity_price_per_kwh = 0.12
natural_gas_price_per_kwh = 10 / kwh_per_mmbtu
cooling_cop = 8.0
heating_cop = 0.85

df_plot["cooling_cost"] = df_plot["cooling_load_kwh"] * electricity_price_per_kwh / cooling_cop * 30
df_plot["heating_cost"] = df_plot["heating_load_kwh"] * natural_gas_price_per_kwh / heating_cop * 30
df_plot["total_cost"] = df_plot["cooling_cost"] + df_plot["heating_cost"]

# for month in df_summary_results["month"].unique():
  # df_month = df_summary_results[df_summary_results["month"] == month]
plt.plot(df_plot["month"], df_plot["cooling_cost"], label=f"Cooling Cost")
plt.plot(df_plot["month"], df_plot["heating_cost"], label=f"Heating Cost")
plt.plot(df_plot["month"], df_plot["total_cost"], label=f"Total Cost")

plt.legend()
plt.title(f"Heating and Cooling Cost for Station {station_id}")
plt.xlabel("Month")
plt.ylabel("Monthly Cost ($/m²)")
plt.show()

In [95]:
from typing import Dict
import numpy as np
import pandas as pd
from pydantic import BaseModel

# Constants
AIR_DENSITY = 1.225  # kg/m³
AIR_SPECIFIC_HEAT = 1005  # J/(kg·K)
JOULE_TO_WH = 1/3600  # conversion factor


class Params(BaseModel):
  u_value: float          # W/m²·K
  height: float          # m
  infiltration_rate: float  # air changes per hour
  thermal_mass: float    # J/m²·K
  glazing_transmittance: float  # fraction of solar radiation transmitted


def calculate_solar_radiation(
  df: pd.DataFrame,
  glazing_transmittance: float
) -> float:
  """Calculate solar radiation for a given hour and day.

  The return value represents the direct solar radiation hitting a
  horizontal surface inside the greenhouse after passing through.

  We account for:
  - The atmospheric attenuation of the sun's rays.
  - The greenhouse glazing/covering transmission losses.
  
  Args:
    hour: float - The hour of the day (0-23)
    day_of_year: int - The day of the year (1-365)
    latitude: float - The latitude of the greenhouse
    glazing_transmittance: float - The transmittance of the glazing
    
  Returns:
    float - The solar radiation in W/m²
  """
  if "hour" not in df.columns:
    raise ValueError("DataFrame must contain a 'hour' column")
  if "day_of_year" not in df.columns:
    raise ValueError("DataFrame must contain a 'day_of_year' column")

  # Solar declination
  declination = 23.45 * np.sin(2 * np.pi * (df["day_of_year"] - 81) / 365)
  
  # Hour angle (15° per hour from solar noon)
  hour_angle = 15 * (df["hour"] - 12)
  
  # Solar altitude
  lat_rad = np.radians(df["latitude"])
  decl_rad = np.radians(declination)
  hour_rad = np.radians(hour_angle)
  
  sin_altitude = (np.sin(lat_rad) * np.sin(decl_rad) + 
                  np.cos(lat_rad) * np.cos(decl_rad) * np.cos(hour_rad))
  solar_altitude = np.arcsin(np.clip(sin_altitude, -1, 1))

  # If the sun is below the horizon, return 0.
  sin_altitude = sin_altitude.max(0)
  
  # Clear sky radiation
  air_mass = 1 / sin_altitude
  
  # Solar constant in W/m²
  solar_constant_w_m2 = 1080
  relative_attenuation_factor = np.exp(-0.1 * air_mass)
  dir_normal = solar_constant_w_m2 * relative_attenuation_factor
  
  # Account for glazing angle
  incident_angle = np.arccos(sin_altitude)
  transmittance = glazing_transmittance * (1 - 0.5 * incident_angle)

  return dir_normal * sin_altitude * transmittance


def simulate_thermal_flux(_df: pd.DataFrame, params: Params, target_temp: float) -> Dict:
  """Simulate entire day of greenhouse operation."""
  df = _df.copy()

  if "temperature" not in df.columns:
    raise ValueError("DataFrame must contain a 'temperature' column")
  
  if "hour" not in df.columns:
    raise ValueError("DataFrame must contain a 'hour' column")

  # Calculate the temperature difference between the target temperature and the temperature outdoors.
  df["delta_t"] = target_temp - df["temperature"]

  # Positive conduction goes from the inside to the outside.
  df["conduction_w_m2"] = params.u_value * df["delta_t"]

  # Positive infiltration goes from the inside to the outside.
  df["infiltration_w_m2"] = params.height * params.infiltration_rate * AIR_DENSITY * AIR_SPECIFIC_HEAT * df["delta_t"]

  # Positive thermal mass goes from the inside to the outside.
  # df["thermal_mass_w_m2"] = params.thermal_mass * df["delta_t"] / 24 * np.sin(2 * np.pi * df["hour"] / 24) * JOULE_TO_WH

  # Passive solar is always positive, and represents heating.
  df["passive_solar_w_m2"] = calculate_solar_radiation(df, params.glazing_transmittance)

  # This is the flux from the outside to the inside.
  df["total_heat_transfer_w_m2"] = (
    df["passive_solar_w_m2"] -
    df["conduction_w_m2"] - 
    df["infiltration_w_m2"]
  )
  
  return df

In [ ]:
df_avg_temp = pd.read_parquet("./ghcn_avg_temp.parquet")

# Shift all hours based on the longitude.
df_avg_temp["hour_offset"] = int(df_avg_temp["longitude"].iloc[0] / 15)
df_avg_temp.loc[df_avg_temp["hour_offset"] < 0, "hour_offset"] = 24 + df_avg_temp["hour_offset"]
df_avg_temp["hour"] = (df_avg_temp["hour"] + df_avg_temp["hour_offset"]) % 24
df_avg_temp.sort_values(by=["Station_ID", "month", "hour"], inplace=True)
df_avg_temp["day_of_year"] = df_avg_temp["month"] / 12 * 365 + 15

station_ids = df_avg_temp["Station_ID"].unique()
print(f"There are {len(station_ids)} unique stations")

params = Params(
  u_value=4.0,            # W/m²·K
  height=4.0,             # m
  infiltration_rate=0.5,  # air changes per hour
  thermal_mass=100000,    # J/m²·K
  glazing_transmittance=0.8,  # fraction
)
target_temp = 23
DAYS_PER_MONTH = 30
SECONDS_PER_HOUR = 3600
W_PER_KW = 1000

df_thermal_flux = simulate_thermal_flux(df_avg_temp, params, target_temp)

df_thermal_flux["hourly_thermal_energy_kwh_m2"] = df_thermal_flux["total_heat_transfer_w_m2"] / W_PER_KW / SECONDS_PER_HOUR
needs_heating_mask = df_thermal_flux["total_heat_transfer_w_m2"] < 0
needs_cooling_mask = df_thermal_flux["total_heat_transfer_w_m2"] > 0

df_thermal_flux["hourly_cooling_load_kwh_m2"] = df_thermal_flux["hourly_thermal_energy_kwh_m2"].abs() * needs_cooling_mask
df_thermal_flux["hourly_heating_load_kwh_m2"] = df_thermal_flux["hourly_thermal_energy_kwh_m2"].abs() * needs_heating_mask

# df_thermal_flux["hourly_cooling_cost_per_m2"] = df_thermal_flux["hourly_cooling_load_kwh_m2"] * electricity_price_per_kwh / cooling_cop
# df_thermal_flux["hourly_heating_cost_per_m2"] = df_thermal_flux["hourly_heating_load_kwh_m2"] * natural_gas_price_per_kwh / heating_cop

df_thermal_flux["cooling_degree_hours"] = df_thermal_flux["delta_t"].abs() * (df_thermal_flux["delta_t"] < 0)
df_thermal_flux["heating_degree_hours"] = df_thermal_flux["delta_t"].abs() * (df_thermal_flux["delta_t"] > 0)
df_thermal_flux["adjusted_cooling_degree_hours"] = df_thermal_flux["delta_t"].abs() * needs_cooling_mask
df_thermal_flux["adjusted_heating_degree_hours"] = df_thermal_flux["delta_t"].abs() * needs_heating_mask

df_thermal_flux

# Add up the total costs for each month by station.
df_total_thermal_costs = df_thermal_flux.groupby(by=["Station_ID", "month"]).agg({
  "hour_offset": "first",
  "latitude": "first",
  "longitude": "first",
  "relative_humidity": "mean",
  # "hourly_cooling_cost_per_m2": "sum",
  # "hourly_heating_cost_per_m2": "sum",
  "cooling_degree_hours": "sum",
  "heating_degree_hours": "sum",
  "adjusted_cooling_degree_hours": "sum",
  "adjusted_heating_degree_hours": "sum",
  "hourly_cooling_load_kwh_m2": "sum",
  "hourly_heating_load_kwh_m2": "sum",
}).reset_index()

# Multiply hourly values by 30 days.
# df_total_thermal_costs["cooling_cost_per_m2"] = df_total_thermal_costs["hourly_cooling_cost_per_m2"] * DAYS_PER_MONTH
# df_total_thermal_costs["heating_cost_per_m2"] = df_total_thermal_costs["hourly_heating_cost_per_m2"] * DAYS_PER_MONTH
df_total_thermal_costs["cooling_load_kwh_m2"] = df_total_thermal_costs["hourly_cooling_load_kwh_m2"] * DAYS_PER_MONTH
df_total_thermal_costs["heating_load_kwh_m2"] = df_total_thermal_costs["hourly_heating_load_kwh_m2"] * DAYS_PER_MONTH
df_total_thermal_costs["cooling_degree_hours"] = df_total_thermal_costs["cooling_degree_hours"] * DAYS_PER_MONTH
df_total_thermal_costs["heating_degree_hours"] = df_total_thermal_costs["heating_degree_hours"] * DAYS_PER_MONTH
df_total_thermal_costs["adjusted_cooling_degree_hours"] = df_total_thermal_costs["adjusted_cooling_degree_hours"] * DAYS_PER_MONTH
df_total_thermal_costs["adjusted_heating_degree_hours"] = df_total_thermal_costs["adjusted_heating_degree_hours"] * DAYS_PER_MONTH
# df_total_thermal_costs["total_cost_per_m2"] = df_total_thermal_costs["cooling_cost_per_m2"] + df_total_thermal_costs["heating_cost_per_m2"]

df_total_thermal_costs.drop(columns=["hourly_cooling_load_kwh_m2", "hourly_heating_load_kwh_m2"], inplace=True)
df_total_thermal_costs.to_parquet("./ghcn_total_thermal_costs.parquet")

df_total_thermal_costs

In [ ]:
def plot_degree_hours(df_plot: pd.DataFrame, degree_hours_column: str, title: str, legend: str):
  # Set matplotlib font:
  plt.rcParams['font.family'] = 'Helvetica Neue'
  fig, ax = plt.subplots(figsize=(12, 5), subplot_kw={'projection': ccrs.PlateCarree()})

  # Add map features
  ax.add_feature(cfeature.COASTLINE)
  ax.add_feature(cfeature.BORDERS, linestyle=':')

  # Set map extent with some padding
  ax.set_extent([
    -180, 180,
    -65, 90
  ])

  df_plot = df_plot[[
    "Station_ID", "latitude", "longitude",
    "heating_degree_hours", "cooling_degree_hours",
    "adjusted_heating_degree_hours", "adjusted_cooling_degree_hours",
    "total_cost_per_m2",
  ]].groupby(by=["Station_ID"]).agg({
    "latitude": "first",
    "longitude": "first",
    "heating_degree_hours": "sum",
    "cooling_degree_hours": "sum",
    "adjusted_heating_degree_hours": "sum",
    "adjusted_cooling_degree_hours": "sum",
    "total_cost_per_m2": "sum",
  }).reset_index()

  # # Remove any station IDs with more than the 95th percentile of heating degree hours.
  # df_plot = df_plot[df_plot["heating_degree_hours"] < df_plot["heating_degree_hours"].quantile(0.95)]
  # # Remove any station IDs with more than the 95th percentile of cooling degree hours.
  # df_plot = df_plot[df_plot["cooling_degree_hours"] < df_plot["cooling_degree_hours"].quantile(0.95)]

  # HEATING DEGREE HOURS
  scatter = ax.scatter(
    df_plot['longitude'],
    df_plot['latitude'],
    c=df_plot[degree_hours_column],
    cmap='RdYlBu_r',
    s=3
  )
  plt.title(f"{title}")
  plt.colorbar(scatter, label=legend)
  fig.show()


kwh_per_mmbtu = 293.07
electricity_price_per_kwh = 0.15
natural_gas_price_per_kwh = 10 / kwh_per_mmbtu
print(f"Electricity price per kWh: ${electricity_price_per_kwh}")
print(f"Natural gas price per kWh: ${natural_gas_price_per_kwh}")
cooling_cop = 8
heating_cop = 0.85
cooling_cost_per_mmbtu = electricity_price_per_kwh / cooling_cop * kwh_per_mmbtu
print(f"Cooling cost per mmbtu: ${cooling_cost_per_mmbtu}")

df_plot = pd.read_parquet("./ghcn_total_thermal_costs.parquet")

# Remove any station IDs with more than the 95th percentile of heating degree hours.
# df_plot = df_plot[df_plot["heating_degree_hours"] < df_plot["heating_degree_hours"].quantile(0.95)]
# Remove any station IDs with more than the 95th percentile of cooling degree hours.
# df_plot = df_plot[df_plot["cooling_degree_hours"] < df_plot["cooling_degree_hours"].quantile(0.95)]

df_plot["heating_cost_per_m2"] = df_plot["heating_load_kwh_m2"] * natural_gas_price_per_kwh / heating_cop
df_plot["cooling_cost_per_m2"] = df_plot["cooling_load_kwh_m2"] * electricity_price_per_kwh / cooling_cop
df_plot["total_cost_per_m2"] = df_plot["heating_cost_per_m2"] + df_plot["cooling_cost_per_m2"]

plot_degree_hours(df_plot, "heating_degree_hours", "Annual Heating-Degree Hours", "degree-hours (celsius)")
# plot_degree_hours(df_plot, "cooling_degree_hours", "Annual Cooling-Degree Hours (Celsius)", "degree-hours (celsius)")
# plot_degree_hours(df_plot, "adjusted_heating_degree_hours", "Adjusted Annual Heating-Degree Hours (Celsius)", "degree-hours (celsius)")
# plot_degree_hours(df_plot, "adjusted_cooling_degree_hours", "Adjusted Annual Cooling-Degree Hours (Celsius)", "degree-hours (celsius)")
# plot_degree_hours(df_plot, "total_cost_per_m2", "Annual energy cost for climate control ($/m²)", "Energy cost ($/m²)")

df_monthly = df_plot[[
  "Station_ID", "month", "latitude", "longitude",
  "heating_degree_hours", "cooling_degree_hours",
  "heating_load_kwh_m2", "cooling_load_kwh_m2",
  "total_cost_per_m2"
]].copy()

# Round values to reduce file size.
for col in ["heating_degree_hours", "cooling_degree_hours", "heating_load_kwh_m2", "cooling_load_kwh_m2", "total_cost_per_m2"]:
  df_monthly[col] = df_monthly[col].round(2)
df_monthly.to_csv("./output/ghcn_monthly_energy_costs.csv", index=False)

df_annual = df_plot.groupby(by=["Station_ID"]).agg({
  "latitude": "first",
  "longitude": "first",
  "heating_degree_hours": "sum",
  "cooling_degree_hours": "sum",
  "heating_load_kwh_m2": "sum",
  "cooling_load_kwh_m2": "sum",
  "total_cost_per_m2": "sum",
}).reset_index()

# Round values to reduce file size.
for col in ["heating_degree_hours", "cooling_degree_hours", "heating_load_kwh_m2", "cooling_load_kwh_m2", "total_cost_per_m2"]:
  df_annual[col] = df_annual[col].round(2)

df_annual[[
  "Station_ID", "latitude", "longitude",
  "heating_degree_hours", "cooling_degree_hours",
  "heating_load_kwh_m2", "cooling_load_kwh_m2",
  "total_cost_per_m2"
]].to_csv("./output/ghcn_annual_energy_costs_full.csv", index=False)

df_annual[["Station_ID", "latitude", "longitude", "heating_degree_hours", "cooling_degree_hours"]].to_csv("./output/ghcn_annual_degree_hours.csv", index=False)
df_annual[["Station_ID", "latitude", "longitude", "total_cost_per_m2"]].to_csv("./output/ghcn_annual_energy_costs.csv", index=False)

In [ ]:
solar_lcoe_price_per_kwh = 0.04
cooling_cost_per_mmbtu = solar_lcoe_price_per_kwh / cooling_cop * kwh_per_mmbtu
print(f"Cooling cost per mmbtu: ${cooling_cost_per_mmbtu}")

Based on hourly NOAA Global Historical Climatology Network (GHCN) data from 2023. Calculated using the difference between the assumed ideal greenhouse temperature of 23°C and the outdoor temperature.

Based on hourly NOAA Global Historical Climatology Network (GHCN) data from 2023.

Assumes an electricity price of $0.15/kWh (COP=8) and a natural gas price of $10/mmbtu (COP=0.85). Models an ideal greenhouse with a U-value of 4 W/m²·K, height of 4 m, infiltration rate of 0.5 air changes per hour, and glazing transmittance of 0.8. Passive solar heating is based on 1080 W/m² of solar radiation and hourly sun angles. Active heating and cooling is applied to maintain the greenhouse at 23°C.